# Get data

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Create index

In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

#if es_client.indices.exists(index=index_name):
#    es_client.indices.delete(index=index_name)


# Health check
es_client = Elasticsearch("http://127.0.0.1:9200")
es_client.info()

# Create index
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course_questions"
es_client.indices.create(index=index_name, body=index_settings)

# Ingest documents
#documents = get_documents()
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:01<00:00, 501.34it/s]


In [7]:
def elastic_search(query: str) -> list:
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        print(f"Score: {hit['_score']}, Document: {hit['_source']}")

    for hit in response["hits"]["hits"][:3]:
        result_docs.append(hit["_source"])
    return result_docs

In [8]:
course = "data-engineering-zoomcamp"
query = "How do execute a command on a Kubernetes pod?"
search_results = elastic_search(query)

Score: 44.50556, Document: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
Score: 35.433445, Document: {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', 'course': 'machine-learning-zoomcamp'}
Score: 33.70974, Document: {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:

In [9]:
def elastic_search(query: str, course: str) -> list:
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []

    for hit in response['hits']['hits']:
        print(f"Score: {hit['_score']}, Document: {hit['_source']}")

    for hit in response["hits"]["hits"][:3]:
        result_docs.append(hit["_source"])
    return result_docs

In [10]:
course = "machine-learning-zoomcamp"
query = "How do copy a file to a Docker container?"
search_results = elastic_search(query, course)

Score: 73.38676, Document: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
Score: 66.688705, Document: {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from

In [12]:
def build_prompt(query: str, search_results: list) -> str:
    context_entries = [
        f"Q: {doc['question']}\nA: {doc['text']}" for doc in search_results
    ]
    context = "\n\n".join(context_entries)

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [13]:
prompt = build_prompt(query, search_results)
len(prompt)

1446

In [15]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

# Encode the prompt into tokens
tokens = encoding.encode(prompt)

# Get number of tokens
num_tokens = len(tokens)
print(f"Token count: {num_tokens}")

Token count: 320
